<a href="https://colab.research.google.com/github/Jumaantony/Django-Blog-App/blob/master/05_Naive_Bayes/Solution_for_lecture_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn.datasets import fetch_20newsgroups
newsgroups_train = fetch_20newsgroups(subset='train')
newsgroups_test = fetch_20newsgroups(subset='test')

In [ ]:
len(newsgroups_train.data)

In [ ]:
newsgroups_train.target, newsgroups_train.target_names

In [ ]:
newsgroups_train.data[0]

In [5]:
def remove_tokens(token_list, text):
    for token in token_list:
        text = text.replace(token, '')
    return text

In [ ]:
from string import punctuation
preprocessed_text = [remove_tokens(punctuation, text) for text in newsgroups_train.data]
  

In [8]:
# Assignemnt one

from sklearn.feature_extraction.text import CountVectorizer
from string import punctuation
preprocessed_text = [remove_tokens(punctuation, text) for text in newsgroups_train.data]
  
num_word = 30000

vectorizer = CountVectorizer(max_features=num_word)
train_data = vectorizer.fit_transform(preprocessed_text)

In [9]:
num_word = train_data.shape[1]

In [11]:
# Assignemnt 2
import numpy as np

class_freq = np.unique(newsgroups_train.target, return_counts=True)
prior_prob = class_freq/np.sum(class_freq)

In [12]:
# Assignemnt 3
data = (train_data > 0).astype('float')
word_label_freq = np.zeros((20, num_word))
for l in range(20):
    ind = newsgroups_train.target == l
    word_label_freq[l] = data[ind].sum(axis=0)

In [ ]:
# Assignment 4


In [ ]:
alpha = 0.01
cond_prob = np.array([(word_label_freq[i] + alpha)/(class_freq[i] + num_word * alpha) for i in range(len(class_freq))])
   

In [21]:

prior_prob.shape

(2, 20)

In [ ]:
# assignment 5

def find_label(data):
    log_prob = np.log(prior_prob)
    data = (data > 0).toarray()
    in_prob = np.log(cond_prob)
    out_prob = np.log(1 - cond_prob)
    log_prob += (in_prob * data + out_prob * (1 - data)).sum(axis=1) 
    return log_prob.argmax()
preprocessed_test_text = [remove_tokens(punctuation, text) for text in newsgroups_test.data]
test_data = vectorizer.transform(preprocessed_test_text)

In [ ]:
pred = []
from tqdm import tqdm
for text in tqdm(test_data):
    pred.append(find_label(text))

In [ ]:
from sklearn import metrics
metrics.accuracy_score(pred, newsgroups_test.target)

In [ ]:
# Assignment 6
word_label_freq = np.zeros((len(newsgroups_train.target_names),train_data.shape[1]))

for i in range(len(newsgroups_train.target)):
  word_label_freq[newsgroups_train.target[i]] += train_data[i]

word_label_freq
print(word_label_freq)

In [ ]:
# Assignment 7
num_word_in_classes = np.sum(word_label_freq,axis=1)
print(num_word_in_classes)
     

In [29]:
# Assignment 8
log_cond_prob = np.array([np.log(word_label_freq[i] + alpha) - np.log(num_word_in_classes[i] + num_word * alpha) for i in range(len(class_freq))])


In [31]:
# Assignment 9
log_prior_prob = np.log(prior_prob)
def find_label(data):
    log_prob = log_prior_prob.copy()
    data = data.toarray()
    log_prob += (log_cond_prob.copy() * data).sum(axis=1)
    return log_prob.argmax()

<ipython-input-31-7afea8c5fc73>:2: RuntimeWarning: divide by zero encountered in log
  log_prior_prob = np.log(prior_prob)


In [ ]:
test_data = vectorizer.transform(preprocessed_test_text)

In [ ]:
pred = []
for text in tqdm(test_data):
    pred.append(find_label(text))
metrics.accuracy_score(pred, newsgroups_test.target)